### Exercise 3.9 Task Part 1

#### 01 Import libraries and files

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'C:\Users\cneva\06-2024 Instacart Basket Analysis'

In [94]:
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

#### 02 Check columns and shape

In [17]:
df_cust.shape

(206209, 10)

In [21]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [96]:
# drop first and last name as this is personal information and probably won't be needed for analysis
df_cust = df_cust.drop(columns = ['First Name', 'Surnam'])

In [98]:
# normalize column names for clarity
df_cust.rename(columns = {'Gender':'gender', 'STATE':'state', 'Age':'age'}, inplace = True)

#### 03 Check high level statistics for anything unusual

In [48]:
df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


#### 04 Check for improperly assigned data types 

In [100]:
df_cust.dtypes

user_id          int64
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [111]:
df_cust['user_id'] = df_cust['user_id'].astype('str')

In [113]:
df_cust.dtypes

user_id         object
gender          object
state           object
age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

#### 05 Check for mixed data types

In [115]:
for col in df_cust.columns.tolist():
    if df_cust[col].apply(type).nunique() > 1:
        print(col)

#### 06 Look for full duplicates

In [129]:
df_cust[df_cust.duplicated()]

,user_id,gender,state,age,date_joined,n_dependants,fam_status,income


#### 07 Look for missing values

In [104]:
df_cust.isnull().sum()

user_id         0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

#### 08 Import Instacart data 

In [127]:
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_4.pkl'))

In [131]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_day_2,busiest_period_of_day,max_order,loyalty_flag,user_average_price,user_spender_type,user_median_days_since,user_order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Mid-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,Low-range product,Regularly busy,Regularly busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [133]:
df_ords_prods.dtypes

order_id                       int64
user_id                        int64
order_number                   int64
orders_day_of_week             int64
order_hour_of_day              int64
days_since_prior_order       float64
first_order                     bool
product_id                     int64
add_to_cart_order              int64
reordered                      int64
product_name                  object
aisle_id                       int64
department_id                  int64
prices                       float64
price_range_loc               object
busiest_day                   object
busiest_day_2                 object
busiest_period_of_day         object
max_order                      int64
loyalty_flag                  object
user_average_price           float64
user_spender_type             object
user_median_days_since       float64
user_order_frequency_flag     object
dtype: object

In [135]:
df_ords_prods['user_id'] = df_ords_prods['user_id'].astype('str')

In [143]:
# check if df_ords_prods have any user_ids that are not in df_cust
df_ords_prods[~df_ords_prods['user_id'].isin(df_cust['user_id'])]['user_id'].count()

0

In [147]:
# check if df_cust have any user_ids that are not in df_ords_prods
df_cust[~df_cust['user_id'].isin(df_ords_prods['user_id'])]['user_id'].count()

0

#### 09 Merge

In [149]:
# merge
df_instacart = df_ords_prods.merge(df_cust, on = 'user_id')

In [150]:
df_instacart.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,user_spender_type,user_median_days_since,user_order_frequency_flag,gender,state,age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [151]:
df_instacart.shape

(32404859, 31)

#### 10 Export 

In [159]:
df_instacart.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust.pkl'))